In [1]:
import csv
import requests
from geopy.distance import geodesic as gd
from datetime import datetime


def convert_into_datetime(trans):
    for i in trans:
        datetime_obj = datetime.strptime(i[-1], "%d/%m/%y %H:%M")
        i[-1] = datetime_obj
    return trans


def dist_bw_location(transactions):
    dist_above_20 = []
    for i in transactions:
        for j in transactions:
            if i == j:
                continue
            if gd(i[3], j[3]).km >= 20 and i[1] == j[1]:
                if ((i, j) not in dist_above_20) and ((j, i) not in dist_above_20):
                    dist_above_20.append((i, j))
    return dist_above_20

def api_request(link):
    response = requests.get(link)

    if response.status_code == 200:
        data = response.json()
        return((data["results"][0]["geometry"]['lat'],
                data["results"][0]["geometry"]['lng']))
    else:
        print("Error:", response.status_code)


def lat_long_convert(transactions):
    # Make a GET request to the API
    for i in transactions:
        PLACE = i[3]
        OPENCAGEDATA_KEY = "7c9e9348137d423180ab7c3b2db55f4e"
        link = f"https://api.opencagedata.com/geocode/v1/json?q={PLACE}&key={OPENCAGEDATA_KEY}"
        i[3] = api_request(link)
    return transactions


def read_csv():
    f = []
    with open("input/q2/transactions.csv") as file:
        lines = csv.reader(file)
        counter = 0
        for line in lines:
            line.insert(0, counter)
            f.append(line)
            counter += 1
    return f

def time_diff(dist_above_20):
    time_30_mins = []

    for i in dist_above_20:
        x = (i[0][4]-i[1][4]).total_seconds()/60
        if x <= 30:
            time_30_mins.append(i[0][0])
            time_30_mins.append(i[1][0])
    return time_30_mins

def fraud(time_30_mins, transactions):
    fraud_transaction = []
    for i in transactions:
        if i[0] in time_30_mins:
            fraud_transaction.append(i[1:])

    # Print
    return fraud_transaction


# Reading CSV file
transac = read_csv()

# Making time into datetime, Longitude & Latitude
transac = convert_into_datetime(transac)

# Convert Distance into Longitude & Latitude
transac = lat_long_convert(transac)

# Checking Distance Between location
dist_above_20 = dist_bw_location(transac)

# Checking Time Difference
time_30_mins = time_diff(dist_above_20)


# Remove counter/id & Print result
print(fraud(time_30_mins, read_csv()))

# Baseline for Enhancement 2.281092 (lowest)

[['4367-7878-8898-8989', 'INR 1000', 'Gurgaon', '27/03/23 17:58'], ['4367-7878-8898-8989', 'INR 5000', 'Noida', '27/03/23 18:10']]
